# Add Cubes Around Surface of Precentral Gyrus

In [1]:
from freesurfer_surface import Surface

SUBJECTS_DIR = '../tests/subjects'
surface = Surface.read_triangular(SUBJECTS_DIR + '/fabian/surf/lh.pial')
surface.load_annotation_file(SUBJECTS_DIR + '/fabian/label/lh.aparc.annot')

precentral_label, = filter(lambda l: l.name == 'precentral', surface.annotation.labels.values())
precentral_label

Label(name=precentral, index=24, color=#3c14dc)

In [2]:
border_chain, = surface.find_label_border_polygonal_chains(precentral_label)
border_chain

PolygonalChain(vertex_indices=(62118, 63255, 63264, 64542, 64562, 65878, 65895, 65925, 65957, 65958, 67222, 67255, 68439, 68458, 69743, 69744, 71109, 72350, 73534, 74843, 76105, 76087, 77297, 77310, 78461, 78462, 79548, 79560, 80703, 81917, 83081, 83082, 84257, 85504, 85505, 86741, 88079, 89444, 90726, 90727, 91981, 91982, 93119, 93120, 93121, 93122, 94229, 94230, 94237, 94238, 94239, 94240, 94241, 94251, 94252, 94265, 95347, 95348, 95337, 96419, 97510, 98721, 98741, 99968, 99979, 99993, 98779, 98778, 98793, 98792, 98807, 98821, 98836, 98854, 97631, 97630, 97609, 97608, 97588, 97587, 97586, 97568, 97585, 96505, 96504, 95443, 95442, 94373, 95456, 95455, 95441, 95425, 95424, 95439, 94343, 94328, 94327, 94326, 94305, 94290, 94280, 93167, 93156, 94270, 94261, 95343, 95334, 96405, 97497, 97496, 97488, 98658, 98641, 99907, 101136, 102349, 102367, 103514, 103537, 103538, 103539, 103540, 104676, 104698, 104699, 105798, 105799, 105818, 106903, 106924, 107998, 107999, 108000, 109016, 109032, 110

In [3]:
import itertools, numpy
from freesurfer_surface import Vertex, Triangle

def slice_offset(iterable, offset):
    return itertools.islice(iterable, offset, len(iterable))

def unit_vector(vector: numpy.array) -> numpy.array:
    return vector / numpy.linalg.norm(vector)

BORDER_WIDTH = 0.8 # mm

for vertex_triplet_indices in zip(*(slice_offset(border_chain.vertex_indices, o) for o in range(3))):
    vertex_triplet_coords = tuple(numpy.array(surface.vertices[idx])
                                  for idx in vertex_triplet_indices)
    backward_vector = vertex_triplet_coords[0] - vertex_triplet_coords[1]
    forward_vector = unit_vector(vertex_triplet_coords[2] - vertex_triplet_coords[1]) * BORDER_WIDTH / 2
    upward_vector = unit_vector(numpy.cross(backward_vector, forward_vector)) * BORDER_WIDTH / 2
    sideward_vector = unit_vector(numpy.cross(upward_vector, forward_vector)) * BORDER_WIDTH / 2
    cross_section_corners = [vertex_triplet_coords[1] + v
                             for v in [-upward_vector -sideward_vector,
                                       -upward_vector +sideward_vector,
                                       +upward_vector +sideward_vector,
                                       +upward_vector -sideward_vector]]
    corner_indices = [[surface.add_vertex(Vertex(*(coords + forward_dir))) for coords in cross_section_corners]
                      for forward_dir in [-forward_vector, +forward_vector]]
    for surface_corner_indices in corner_indices:
        surface.triangles.append(Triangle(surface_corner_indices[:3]))
        surface.triangles.append(Triangle(surface_corner_indices[2:] + surface_corner_indices[:1]))
    for i in range(4):
        j = (i + 1) % 4
        surface.triangles.append(Triangle((corner_indices[0][i], corner_indices[0][j], corner_indices[1][j])))
        surface.triangles.append(Triangle((corner_indices[0][i], corner_indices[1][j], corner_indices[1][i])))

surface.write_triangular('precentral-border.lh.pial')

```sh
freeview --surface examples/precentral-border.lh.pial:annot=tests/subjects/fabian/label/lh.aparc.annot
```